<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

# Swan Consulting Predictive Modelling
### <i>By Team Ugly Ducklings</i>

### Table Of Contents:
* [Data Cleaning & Feature Engineering](#1)
* [Create X_train, X_test, y_train, and y_test](#2)
* [Predictive Modelling](#3)
* [Create Required Lists](#4)

The aim of this workbook is to create a predictive model to output a list of those most likely to churn next, and a list for the churn risk.

In [1]:
## Imports
## Importing the big 4 - Pandas, Numpy, Seaborn & matplotlib
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
## Import the metrics we'll be using
from sklearn import metrics
## Import Logistic Regression from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
pd.options.display.max_columns = None

In [3]:
df = pd.read_csv("swan_data.csv")

## Data Cleaning & Feature Engineering <a class="anchor" id="1"></a>

In [4]:
def data_cleaning(df):
    #Making a copy of the dataset
    df = df.copy()

    #Using CustomerID as index
    df.set_index("CustomerID", inplace=True)

    #Fixing Total Charges column - inserting zeroes for blank columns and casting to float
    df["Total Charges"] = df["Total Charges"].str.replace(" ", "0")
    df = df.astype({"Total Charges": float})
    
    #Scale Total Charges, Monthly Charges, and Tenure to be closer to values in other columns
    df['Tenure_scaled'] = df['Tenure Months'] / 100
    df['Monthly Charges_scaled'] = df['Monthly Charges'] / 100
    df['Total Charges_scaled'] = df['Total Charges'] / 1000
    
    #Dropping unnecessary columns
    df.drop(columns=["Count", "Country", "State", "City", "Zip Code", "Lat Long", "Churn Label", "Churn Reason"], inplace=True)

    #Mapping columns to numeric values
    #General case, where values are "yes" and "no". Results like "no phone service" are mapped to 0 since the lack of phone service is contained in a different column
    general_mapper = {"No":0, "Yes":1, "No phone service": 0, "No internet service": 0}
    for col in df.columns:
        if "No" in df[col].unique() and "Yes" in df[col].unique():
            df[col] = df[col].map(general_mapper)
    
    #Mapping male to 0 and female to 1
    gender_mapper = {"Male":0, "Female":1}
    df["Gender"] = df["Gender"].map(gender_mapper)

    #Mapping contract lengths into a value representing the length of the contract term in years
    contract_mapper = {"Month-to-month": 1/12, "Two year": 2, "One year":1}
    df["Contract"] = df["Contract"].map(contract_mapper)

    #One Hot Encoding the internet service and payment method columns
    df = pd.get_dummies(df, columns=["Internet Service"], dtype=int, prefix="is", drop_first=True)
    df = pd.get_dummies(df, columns=["Payment Method"], dtype=int, prefix="pay", drop_first=True)

    return df


In [5]:
df = data_cleaning(df)

In [6]:
df.head()

,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Monthly Charges,Total Charges,Churn Value,Tenure_scaled,Monthly Charges_scaled,Total Charges_scaled,is_Fiber optic,is_No,pay_Credit card (automatic),pay_Electronic check,pay_Mailed check
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3668-QPYBK,33.964131,-118.272783,0,0,0,0,2,1,0,1,1,0,0,0,0,0.083333,1,53.85,108.15,1,0.02,0.5385,0.10815,0,0,0,0,1
9237-HQITU,34.059281,-118.307420,1,0,0,1,2,1,0,0,0,0,0,0,0,0.083333,1,70.70,151.65,1,0.02,0.7070,0.15165,1,0,0,1,0
9305-CDSKC,34.048013,-118.293953,1,0,0,1,8,1,1,0,0,1,0,1,1,0.083333,1,99.65,820.50,1,0.08,0.9965,0.82050,1,0,0,1,0
7892-POOKP,34.062125,-118.315709,1,0,1,1,28,1,1,0,0,1,1,1,1,0.083333,1,104.80,3046.05,1,0.28,1.0480,3.04605,1,0,0,1,0
0280-XJGEX,34.039224,-118.266293,0,0,0,1,49,1,1,0,1,1,0,1,1,0.083333,1,103.70,5036.30,1,0.49,1.0370,5.03630,1,0,0,0,0


In [7]:
df.columns

Index(['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Monthly Charges', 'Total Charges', 'Churn Value', 'Tenure_scaled',
       'Monthly Charges_scaled', 'Total Charges_scaled', 'is_Fiber optic',
       'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check'],
      dtype='object')

In [8]:
df.shape

(7043, 28)

## Create X_train, X_test, y_train, and y_test <a class="anchor" id="2"></a>

### Experimenting with 

In [9]:
feature_cols = ['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Monthly Charges', 'Total Charges', 'is_Fiber optic',
       'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check']
# Test scores (prior to any scaling)
# Accuracy:0.7444391859914813
# Precision:0.525974025974026
# Recall:0.826530612244898
# F1:0.6428571428571429

In [10]:
feature_cols = ['Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Monthly Charges', 'Total Charges', 'is_Fiber optic',
       'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check']

# Test scores (prior to any scaling)
# Accuracy:0.7444391859914813
# Precision:0.5262008733624454
# Recall:0.8197278911564626
# F1:0.6409574468085106

In [11]:
feature_cols = ['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Contract', 'Paperless Billing',
       'is_Fiber optic', 'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check',
       'Tenure_scaled', 'Monthly Charges_scaled', 'Total Charges_scaled', ]
# Accuracy:0.7482252721249408
# Precision:0.5309734513274337
# Recall:0.8163265306122449
# F1:0.64343163538874

In [500]:
feature_cols = ['Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Contract', 'Paperless Billing',
       'is_Fiber optic', 'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check',
       'Tenure_scaled', 'Monthly Charges_scaled']

In [250]:
feature_cols = ['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'is_Fiber optic', 'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check',
       'Tenure_scaled', 'Monthly Charges_scaled', 'Total Charges_scaled', ]

# Test scores
# Accuracy:0.7439659252247989
# Precision:0.5256270447110142
# Recall:0.8197278911564626
# F1:0.640531561461794

In [307]:
feature_cols = ['Contract', 'Paperless Billing',
       'is_Fiber optic', 'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check',
       'Tenure_scaled', 'Monthly Charges_scaled', 'Total Charges_scaled', ]
# Accuracy:0.7236157122574538
# Precision:0.5020964360587002
# Recall:0.814625850340136
# F1:0.6212710765239948

In [359]:
feature_cols = ['Contract',
       'is_Fiber optic', 'is_No', 
       'Tenure_scaled', 'Monthly Charges_scaled', 'Total Charges_scaled', ]
# Accuracy:0.7122574538570753
# Precision:0.48973305954825463
# Recall:0.8112244897959183
# F1:0.6107554417413572


In [382]:
feature_cols = ['Contract',
       'is_Fiber optic', 'is_No', 
       'Tenure_scaled' ]
# Accuracy:0.7127307146237577
# Precision:0.49019607843137253
# Recall:0.8078231292517006
# F1:0.6101477199743095

#### Final Decision

In [12]:
###### USE THIS ONE #########
feature_cols = ['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'is_Fiber optic', 'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check',
       'Tenure_scaled', 'Monthly Charges_scaled', 'Total Charges_scaled', ]
# Accuracy:0.7477520113582584
# Precision:0.5301866081229418
# Recall:0.8214285714285714
# F1:0.6444296197464976

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], #X
                                                    df['Churn Value'], #y
                                                    test_size = 0.3, 
                                                    random_state = 42)

## Predictive Modelling <a class="anchor" id="3"></a>

In [14]:
## one function for accuracy, precision, and recall

def apr(y_pred, y_real):
    accuracy = metrics.accuracy_score(y_real, y_pred)
    precision = metrics.precision_score(y_real, y_pred)
    recall = metrics.recall_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred)
    
    print(f"Accuracy:{accuracy}")
    print(f"Precision:{precision}")
    print(f"Recall:{recall}")
    print(f"F1:{f1}")
    return accuracy, precision, recall, f1


## one function for confusion matrix

def produce_confusion(positive_label, negative_label, cut_off, df, y_pred_name, y_real_name):
    
    #Set pred to 0 or 1 depending on whether it's higher than the cut_off point.
    
    if cut_off != 'binary':      
        df['pred_binary'] = np.where(df[y_pred_name] > cut_off , 1, 0)
    else: 
        df['pred_binary'] = df[y_pred_name]
    
    #Build the CM
    cm = confusion_matrix(df[y_real_name], df['pred_binary'])  
    
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax=ax, fmt='g'); 

    # labels, title, ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('Real labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels([negative_label, positive_label])
    ax.yaxis.set_ticklabels([negative_label, positive_label]);

    print('Test accuracy = ', accuracy_score(df[y_real_name], df['pred_binary']))

    return accuracy_score(df[y_real_name], df['pred_binary'])

In [15]:
# create and fit logistic regression model
lr = LogisticRegression(max_iter=500, random_state=10, class_weight='balanced')
lr.fit(X_train, y_train)

c:\Users\matth\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=500, random_state=10)

In [16]:
# predict on X_train
# create new columns to store the probabilities and predictions
X_train = X_train.copy()
X_train[['prob_not_churn', 'prob_churn']] = lr.predict_proba(X_train)
X_train['y_pred'] = np.where(X_train['prob_churn']>.5, 1, 0)

# show metrics
apr(X_train['y_pred'], y_train)

Accuracy:0.7592292089249493
Precision:0.5235235235235235
Recall:0.8165495706479313
F1:0.6379993900579445


(0.7592292089249493,
 0.5235235235235235,
 0.8165495706479313,
 0.6379993900579445)

## On Test Data 

In [17]:
# predict on X_test
# create new columns to store the probabilities and predictions
X_test = X_test.copy()
X_test[['prob_not_churn', 'prob_churn']] = lr.predict_proba(X_test)
X_test['y_pred'] = np.where(X_test['prob_churn']>.5, 1, 0)

# show metrics
apr(X_test['y_pred'], y_test)

Accuracy:0.7477520113582584
Precision:0.5301866081229418
Recall:0.8214285714285714
F1:0.6444296197464976


(0.7477520113582584,
 0.5301866081229418,
 0.8214285714285714,
 0.6444296197464976)

## Create Required Lists <a class="anchor" id="4"></a>

In [18]:
# create a copy of the full dataset
df_mod = df.copy()

In [19]:
df_mod.columns

Index(['Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner',
       'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Monthly Charges', 'Total Charges', 'Churn Value', 'Tenure_scaled',
       'Monthly Charges_scaled', 'Total Charges_scaled', 'is_Fiber optic',
       'is_No', 'pay_Credit card (automatic)', 'pay_Electronic check',
       'pay_Mailed check'],
      dtype='object')

In [20]:
# apply model to full dataset
# create new columns to store the probabilities and predictions
df_mod[['Not Churn Probability', 'Churn Probability']] = lr.predict_proba(df_mod[feature_cols])   # predict probabilites for not churning and churning
df_mod['y_pred'] = np.where(df_mod['Churn Probability']>.5, 1, 0)                                 # predictions based on a threshold of 0.5

In [21]:
df_mod.head()

,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Monthly Charges,Total Charges,Churn Value,Tenure_scaled,Monthly Charges_scaled,Total Charges_scaled,is_Fiber optic,is_No,pay_Credit card (automatic),pay_Electronic check,pay_Mailed check,Not Churn Probability,Churn Probability,y_pred
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3668-QPYBK,33.964131,-118.272783,0,0,0,0,2,1,0,1,1,0,0,0,0,0.083333,1,53.85,108.15,1,0.02,0.5385,0.10815,0,0,0,0,1,0.379998,0.620002,1
9237-HQITU,34.059281,-118.307420,1,0,0,1,2,1,0,0,0,0,0,0,0,0.083333,1,70.70,151.65,1,0.02,0.7070,0.15165,1,0,0,1,0,0.374652,0.625348,1
9305-CDSKC,34.048013,-118.293953,1,0,0,1,8,1,1,0,0,1,0,1,1,0.083333,1,99.65,820.50,1,0.08,0.9965,0.82050,1,0,0,1,0,0.303831,0.696169,1
7892-POOKP,34.062125,-118.315709,1,0,1,1,28,1,1,0,0,1,1,1,1,0.083333,1,104.80,3046.05,1,0.28,1.0480,3.04605,1,0,0,1,0,0.435244,0.564756,1
0280-XJGEX,34.039224,-118.266293,0,0,0,1,49,1,1,0,1,1,0,1,1,0.083333,1,103.70,5036.30,1,0.49,1.0370,5.03630,1,0,0,0,0,0.647646,0.352354,0


In [22]:
# filter for those that have not yet churned
not_churned = df_mod[df_mod['Churn Value'] == 0]

In [23]:
# sort 'Churn Probability' values in descending order and write their indexes to a list in a text file
my_file = open("most_likely_churners.txt", "w")
my_file.write(f"{list(not_churned.sort_values('Churn Probability', ascending=False).index[:500])}")
my_file.close()

In [144]:
# Open the file in write mode
with open("churn_risk.txt", "w") as my_file:
    # Iterate over the DataFrame rows using iterrows() to get index and row data
    for customer_id, row in not_churned.iterrows():
        # Write the customer ID and churn probability to the file
        my_file.write(f"Customer ID: {customer_id}, Churn Probability: {row['Churn Probability']}\n")

In [26]:
not_churned["Churn Probability"].to_csv("churn_risk.csv")